# Coleccion de Funciones

In [1]:
import base64
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from sklearn import preprocessing


# Codificar valores de texto en variables ficticias

def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Codifique valores de texto en una sola variable ficticia. 
# Las nuevas columnas (que no reemplazan a las antiguas) tendrán un '1' 
# en cada ubicación donde la columna original

def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = f"{name}-{tv}"
        df[name2] = l


# Codifica valores de texto en índices

def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Codifica una columna numérica como zscores

def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convierte todos los valores faltantes en la columna específica en la mediana

def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convierte todos los Na en la columna especifica en al valor predeterminado

def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convierte un dataframe de Pandas a las entradas x,y que necesita TensorFlow

def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # Averigüe el tipo de la columna de destino. ¿Es realmente tan difícil? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Transforma a int para la clasificación o flot de lo contrario. A TensorFlow le gustan los 32 bits.
    if target_type in (np.int64, np.int32):
        # Clasificación
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regresión
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

# Cadena de time bien formateada

def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"


# Regresión chart.

def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred': pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='expected')
    plt.plot(t['pred'].tolist(), label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Elimina todas las filas donde la columna especifica es +/- desviaciones estándar sd
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Codifica una columna en un rango entre normalized_low y normalized_high.

def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
        * (normalized_high - normalized_low) + normalized_low
